In [15]:
import string
import itertools
import os
import subprocess
import random
import string
import re
import hashlib

# up to pattern size of 19, dunno why

string_length = 1024 * 1024 * 1024
def randomString(stringLength=256, subject=True):
    """Generate a random string of fixed length """
    letters = "ab"
    if subject:
        chunk = 1024*1024*128 # 128MB
        if os.path.exists("./data/subject.txt"):
          os.remove("./data/subject.txt")
        with open("./data/subject.txt","a+") as s:
            for j in range(0,int(((stringLength+chunk-1)/chunk))):
                    rightBound = (j+1)*chunk if (j+1) * chunk < stringLength else stringLength
                    print(rightBound)
                    randomStr = ''.join(random.choice(letters) for i in range(j*chunk,rightBound))
                    s.write(randomStr)
        return
    else:
        randomStr =  ''.join(random.choice(letters) for i in range(stringLength))
        with open("./data/pattern.txt","w") as s:
            s.write(randomStr)
        return randomStr


files = [
            "dirty_chunk.txt",
         "dirty_nochunk.txt",
#          "kmp_pe_chunk.txt",
#          "kmp_pe_nochunk.txt",
#          "dirty_naive_chunk.txt",
#          "dirty_naive_nochunk.txt",
#          "clean_pe_chunk.txt",
#          "clean_pe_nochunk.txt",
#          "naive_chunk.txt",
#          "naive_nochunk.txt",
#          "kmp_chunk.txt",
#          "kmp_nochunk.txt",
         "const_nochunk.txt",
         "shared_nochunk.txt"]

def match(pattern):
    commands = {
    "dirty_chunk" : f"./fun {pattern} dirty 0 > dirty_chunk.txt",
    "dirty_nochunk" : f"./fun {pattern} dirty 1 > dirty_nochunk.txt",
#     "kmp_chunk" : f"./fun {pattern} kmp 0 > kmp_pe_chunk.txt",
#     "kmp_nochunk" : f"./fun {pattern} kmp 1 > kmp_pe_nochunk.txt",
#     "dirty_naive_chunk" : f"./fun {pattern} dirtynaive 0 > dirty_naive_chunk.txt",
#     "dirty_naive_nochunk" : f"./fun {pattern} dirtynaive 1 > dirty_naive_nochunk.txt",
#     "naive_clean_chunk" : f"./fun {pattern} clean 0 > clean_pe_chunk.txt",
#     "naive_clean_nochunk" : f"./fun {pattern} clean 1 > clean_pe_nochunk.txt",
    
#     "naive_chunk" : f"./clean.out {pattern} naive 0 > naive_chunk.txt",
#     "naive_nochunk" : f"./clean.out {pattern} naive 1 > naive_nochunk.txt",
    
#     "kmp_naive_chunk" : f"./clean.out {pattern} kmp 0 > kmp_chunk.txt",
#     "kmp_naive_nochunk" : f"./clean.out {pattern} kmp 1 > kmp_nochunk.txt",
        
    "naive_const" : f"./clean.out {pattern} const 1 > const_nochunk.txt",
    "naive_shared" : f"./clean.out {pattern} naivesh 1 > shared_nochunk.txt"
        
        
    }
    
    for command in commands.values():    
        os.system(command)

def matchNaive(pattern):
    command = f"./fun {pattern} clean 0 > clean_pe_chunk.txt"
    os.system(command)

#     l = f.readlines()
#     return l[-1][:-1]
def matchNaiveDevice(pattern):
    command = f"./clean_naive_pointer.out {pattern} > outputNaiveDevice.txt"
    os.system(command)
            
                

def generate_strings(length=3):
#     chars = string.ascii_letters + string.digits  # "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    chars = 'ab'
    for item in itertools.product(chars, repeat=length):
        yield "".join(item)
       
# cartesian product of input iterables
# strings = [''.join(x) for x in itertools.product('abcdefghijklmnopqrstuvwxyz', repeat=20)]
# patterns = [''.join(x) for x in itertools.product('abcdefghijklmnopqrstuvwxyz', repeat=6)]

strings = generate_strings(15)
patterns = generate_strings(5)

def test_correctness(f):
    for num in range(6,10):
        strings = generate_strings(num)
        for i in strings:
            for num2 in range(1,5):
                patterns = generate_strings(num2)
                for j in patterns:
                    res1 = match(j,i)
                    res2 = match2(j,i,f)
                    if(res1 == res2):
                        continue
                    else:
                        error = f"Test failed on: pattern = {j} string = {i}"
                        print(error)
                        return
    print("Test passed")

# with open("output.txt") as f:
#     test(f)

def all_equal(iterable):
    "Returns True if all the elements are equal to each other"
    g = itertools.groupby(iterable)
    return next(g, True) and not next(g, False)

def test():
    outputs = []
    for file in files:
        with open(file) as f:
            l = f.readlines()
            hash_object = hashlib.sha256(l[-1][:-1].encode('utf-8'))
            hex_dig = hash_object.hexdigest()
            outputs.append(hex_dig)
    return(all_equal(outputs))
        
        
def time():
    rePatt = re.compile('running time [\d]*\.?[\d]*')
    result = []
    for file in files:
        with open(file) as mN:
            for line in mN:
                if (rePatt.findall(line) != []):
                    result.append(line)
                    break
    return(result)

def time_naive():
    rePatt = re.compile('running time [\d]*\.?[\d]*')
    result = []
    with open("clean_pe_chunk.txt") as mN:
        for line in mN:
            if (rePatt.findall(line) != []):
                result.append(line)
                break
    return(result)


    
# pattern = randomString(19,False)
# print(pattern)
# randomString(1024*1024*100) #if size > 19 PE aborts

# matchPE("sunmnnctxnvkowicoe")
# matchNaive("sunmnnctxnvkowicoe")
# print(test())

rePatt = re.compile('[\d]+\.?[\d]*')
# for line in time():
#     print(rePatt.findall(line))

def checkTimeDynamic(size):
    res = []
    for _ in range(0,50):
        pattern = randomString(18,False)
        randomString(1024*1024*size) #if size > 19 PE aborts
        matchPE(pattern)
        matchNaiveDevice(pattern)
        
        if(test()):
            clocks = []
            for line in time():
                clocks.append(rePatt.findall(line))
            entry = (pattern,clocks[0][0],clocks[1][0]) #first time is Naive 2nd is PE'ed
            res.append(entry)
        else:
            print("Match failed " + pattern)
            return
    return res

def checkTimeStatic(size):
    res = []
    randomString(1024*1024*size)
    for _ in range(0,10):
        pattern = randomString(16,False) #if size > 19 PE aborts
        for _ in range(2):
            match(pattern) #prerun for jit
#         matchNaiveDevice(pattern)
#         if(test()):
        clocks = []
        for line in time():
            clocks.append(rePatt.findall(line))
        entry = (pattern,[x[0] for x in clocks]) #first time is Naive 2nd is PE'ed
        res.append(entry)
#         else:
#             print("Match failed " + pattern)
#             return
    return res


def checkTimeStaticNaive(size):
    res = []
    pattern = randomString(18,False)
    for _ in range(0,50):
#         pattern = randomString(18,False)
        randomString(1024*1024*size) #if size > 19 PE aborts
        matchNaive(pattern)
        clocks = []
        for line in time_naive():
            clocks.append(rePatt.findall(line))
        entry = (pattern,clocks[0][0]) #first time is Struct 2nd is Device
        res.append(entry)
    return res

testDynamic1MB = [('gwzuuzkmpepyzmiuzl', '1', '289'), ('stsjamorvirnenhhme', '1', '341'), ('vhqmahkvehaokdqbwf', '1', '325'), ('khahkmmlpsamfemfnf', '1', '344'), ('iexevvyhrgwzuestfo', '1', '345'), ('ozuvgaybwxeqijglln', '1', '316'), ('ipialicukvnhrhrvlz', '1', '353'), ('oueiktjtzcyjswppsa', '1', '510'), ('tadzykycbdswjriiew', '1', '360'), ('qnxraptkmpeyqfehcf', '1', '413'), ('fboyufcbnxvsoxthnu', '1', '336'), ('eprpcevbmvwbbkallm', '1', '333'), ('zfshhlxqsbnxjtbxai', '1', '355'), ('kkvtqfggtlwdnhtyir', '1', '320'), ('fdzaxyuuqohykelbca', '1', '320'), ('gztksqzvnoefzrutdj', '1', '478'), ('wsfjmhjbgyqfatlvac', '1', '618'), ('ccpijuxwcmdojuqomv', '1', '374'), ('pzjkqxllabyjezxkso', '1', '344'), ('cdgcfwqewixmvwrihu', '1', '505'), ('retrorywgxqhthwevv', '1', '415'), ('erlefpbnvdcpuenbke', '1', '367'), ('kbnrekihgjqukkzlpm', '1', '573'), ('clyuiodeinbozbblgf', '1', '318'), ('sksigexnncfiogksai', '1', '428'), ('rqvxahwhiqiqsnkhpo', '1', '318'), ('zjoyfufuamigxggdua', '1', '315'), ('qkhqeqpvhyolxqolzc', '1', '312'), ('hfjgdwljkqospeejsh', '1', '301'), ('hoccnczrmiqrwbtsij', '1', '321'), ('katflmoilpnlitkkfl', '1', '308'), ('giknhdytfortqbivlz', '1', '316'), ('tpmtqorarhhlmfajsu', '1', '312'), ('ojfguzalvbyiiddaip', '1', '318'), ('gyonrlnyodxzsvdblx', '1', '314'), ('hlvezjcgowzvdrxlfc', '1', '305'), ('boxtvvjwxnddkbjzgh', '1', '316'), ('hsgcndfzkorxijijqp', '1', '309'), ('sflcbkcjqlaosnirtf', '1', '393'), ('qgbwtbvviuavaettwh', '1', '303'), ('tcmdpbgsteznhgzwgp', '1', '312'), ('xcjftblinsyguhrgcv', '1', '312'), ('zyqubhzcxrbiubactv', '1', '319'), ('yptjdkdkudkdvpfuhz', '1', '302'), ('muylnljyjgrqapvtis', '1', '340'), ('uivclkpnbigjqakyrx', '1', '317'), ('nuzmbzyivkfekghtqh', '1', '300'), ('oclwfvwgefagaouqhb', '1', '292'), ('rckugfewhlcjgtrvdk', '1', '320'), ('venzfhyrlypejxydbf', '1', '312')]
testDynamic10MB = [('bzrmyxceuonfjqpqcl', '13', '273'), ('whgzmnumnhpuqkndfg', '13', '295'), ('quuqgoqdiahxnuqjqr', '13', '319'), ('vqchnvwjrgjrfvjnpf', '13', '308'), ('enndzzraflosbrhjal', '13', '387'), ('ykioebyikptbpmlnyn', '13', '301'), ('buzbddhdwyzgamedqq', '13', '305'), ('tkbfseqaguycgmbomm', '13', '300'), ('svskzphzpnfhwdozpn', '13', '303'), ('ctgsrltwkozazprxsj', '13', '301'), ('yiasbssmmumiepurfn', '13', '313'), ('rorrkdltjvrdsmxfsk', '13', '303'), ('ecmmomojcanotglkjg', '13', '297'), ('qzlspmyzfxyfibakfu', '13', '317'), ('lgqysyqussuuzhiobd', '13', '310'), ('vhweykjtjtqhsdwnbq', '13', '312'), ('yimjnesjkiwwvqulgv', '13', '294'), ('bvjtdselsacxjasxfp', '13', '304'), ('wikapvgqxkvbzeehcx', '13', '314'), ('wbvneufgxztwkwugfe', '13', '354'), ('zbxkrtxjggqedilwbo', '13', '395'), ('fnjpctdmmjifwrkkqr', '13', '399'), ('thwreohyvrdcsqxwrz', '13', '481'), ('yskwtirvheknofboyk', '13', '310'), ('aogmaoqnzryrspumhk', '13', '708'), ('zmswkptbubvxhiqgru', '12', '545'), ('qqkkqswzseqxomjnec', '13', '522'), ('irkpecqkwsyttvhext', '13', '495'), ('yxinownboybacduerk', '13', '494'), ('phfdsedssswhfukmvz', '13', '536'), ('tuurqwfduorxgbycsr', '13', '532'), ('rsgehyofrwtvpqxnwf', '13', '546'), ('bjlhssplqkakdabiff', '13', '486'), ('mtfwatdxjygzojkjuj', '13', '481'), ('jrzbkxxjiyohfirghj', '13', '356'), ('enrkdyanzeulkpenfq', '13', '552'), ('ovdljydyutbnxcxpvo', '13', '389'), ('gbikxrsvevvetwgkox', '13', '355'), ('boxrxbcxvyftfmqlri', '13', '406'), ('hujmfvkqhvkzhugzre', '13', '562'), ('vznzpkblfqghyxhmyh', '13', '351'), ('gnoldcnkcqpflnpvrh', '13', '394'), ('usyfmjzhajpcqkwudx', '13', '473'), ('qdoueyroxjmdmidheu', '13', '509'), ('hpiczblukzlpuvxevt', '13', '318'), ('uadaxizwjnuoartqqc', '13', '376'), ('hfqudkosecldbtimup', '13', '337'), ('iegnpuhjatbbitavpk', '13', '370'), ('fkshgitvzwkglbcxzr', '12', '338'), ('zskgxprrriilnnsnyp', '13', '786')]
testDynamic50MB = [[('wyfopvpnoketjglluj', '65', '1792'), ('gazahmmricbxfvrukj', '64', '373'), ('kvpdzeuzbimiogsbvb', '65', '741'), ('wlcwknzgjmhlcqlzde', '64', '400'), ('wniwdazenoqzambljz', '64', '340'), ('fgvfkntpnydepytkvo', '64', '318'), ('apeqxtxahzqgjtzlen', '64', '343'), ('ywbgrpclcmkwxzvdzx', '64', '357'), ('wksqyjdfbrxbgvfwzs', '64', '493'), ('nujoqouvynjxqpbnga', '65', '433'), ('azhtazfbcfiokklquk', '64', '797'), ('wmeagkaczdgiftcibi', '64', '407'), ('qseepzaigjmujhncxp', '64', '541'), ('yxequkzqhandxufyey', '64', '421'), ('yqrjuaybflerxnpzgf', '65', '668'), ('eleayxdtggraeftzzo', '64', '442'), ('dbvzconbhyruynbsvt', '64', '425'), ('hzyttdpwvxgrhotske', '65', '752'), ('ibdabdpmqsswinlalg', '64', '345'), ('tlwxnoyhufvmdkaxmv', '64', '444'), ('afakveriwsrsqlhtzm', '65', '737'), ('jwxbsuyjlvivfwboos', '64', '524'), ('ltzmckdwsgkbvulnbn', '64', '451'), ('ndjhllhtizgcqfapfx', '64', '307'), ('xnvmuannwmyrqrnzhx', '65', '586'), ('ddfftwodwyhkklluee', '64', '298'), ('lggcxtwwwrgxsuahyw', '64', '341'), ('vbzqffqhrsfebxrmga', '64', '290'), ('vaqaryujfyhqyjfnks', '64', '279'), ('pcmibxsnvsartrjaxt', '64', '288'), ('dqxhfcszycjdioidwh', '64', '299'), ('vjqlerefcpimjakogm', '64', '388'), ('gthvtzqnklfpppqbyz', '64', '524'), ('rygwpsbshpavqhjkvu', '64', '462'), ('detjilrqgcgjdkcpry', '64', '508'), ('nagymbzonhygmjifkj', '64', '565'), ('dslutznfqbfjfstdqu', '64', '426'), ('tdjgtxrpnrmjebhtxo', '65', '412'), ('suiurvyieyyyojmyho', '64', '432'), ('vefxzbqfelctmvcjin', '64', '465'), ('etcymwtpckeyepomwp', '64', '427'), ('vnxahhjxywjoqlkumi', '64', '407'), ('xjbcgjbuytevvwhuyb', '65', '428'), ('svztgsrsttzjgnoqbf', '64', '746'), ('jqsnwpwcttlkzjcrgt', '64', '489'), ('saqoggvuvsnzxgifnl', '64', '466'), ('dpmcwmwmnopoqhrrvv', '65', '423'), ('kzazmcchyzbwjbdifp', '64', '560'), ('eguzxnudziscmzuyoy', '64', '722'), ('ixwocqsrfzustdcqjr', '64', '440')]]
unknown_error = ["ugggzpqxckkodstevw"]
testDynamic100MB = [('pffwjardydzayszpts', '129', '344'), ('zvwnpxnhwxvaligvox', '129', '290'), ('jrxzcvrfobxtksoyro', '129', '303'), ('ijkcwiwwhoeaamcudz', '129', '442'), ('otxlrjpezasvzecevr', '129', '605'), ('knwwvdkpiufttclodw', '129', '434'), ('iomcehyezxwwzimrty', '129', '438'), ('bhpuaxbepwchebjgfe', '129', '679'), ('jclejgmuyandjetcnq', '129', '354'), ('vemchpehhswlrkrnuu', '129', '307'), ('nrscqxdqpbjtlzvrfv', '129', '376'), ('xpjevxpthefoetcuvz', '129', '765'), ('mbwmqwxzxejwwopybb', '129', '393'), ('xptsvecbzkkrrmovuj', '129', '465'), ('tmysgwnofihisiujby', '129', '297'), ('dqocenfgipzixgbgng', '129', '427'), ('xkdnupdrpmzndzpimn', '129', '368'), ('ludmkbkcqisxtarquf', '129', '298'), ('vygwuzsrumglhhqlsq', '129', '315'), ('slvetbryacxgnpfrye', '129', '305'), ('japxvywzpexjsjnvot', '129', '297'), ('xlifmxpcyujnqigsax', '129', '304'), ('pztvfqhufwdcdvwmqn', '129', '300'), ('icsxlbdbunbijxjcnt', '129', '353'), ('dmtmsjlejhfytmstzk', '129', '288'), ('tysjdfcjqltdpqfibw', '129', '286'), ('ltjhmhldcljprgkzbc', '129', '292'), ('qdibkbxdhrlvgjvpxt', '129', '340'), ('reqpfstygtnjtxhbcq', '129', '296'), ('meqeciizamvziuzqmg', '129', '287'), ('ujbdgmlooemquxulae', '130', '301'), ('hyrihvunwjuqvztfxw', '129', '738'), ('pflzrdbvfvtqtqygsm', '129', '930'), ('khmzcfwkfawkntofuc', '129', '586'), ('lyjhkeiodbnrqexood', '129', '1253'), ('uljxvpiplfmoxmibbv', '129', '617'), ('gtilqlqxitbtsgizwd', '130', '708'), ('ldqkboeofiscnmdnzy', '129', '684'), ('yxwjqyfheglfezlkqq', '129', '693'), ('bkpwihpnoculkazaqb', '130', '708'), ('ysalwuogjtqaoarxqy', '129', '633'), ('fcvqpkpxmmdqhusuiq', '129', '368'), ('rtnifpjanzepddpxey', '129', '724'), ('aeyuudxcgcdguvpyuw', '129', '684'), ('hqedjmyxaadcrfkbru', '129', '611'), ('qjdgvfxwuyaxkbeiux', '130', '749'), ('oayipbqhdbpqdikjfn', '129', '714'), ('gxavvorucddmkbngia', '129', '666'), ('wbgssuxklxbkjraohh', '129', '344'), ('ybkejmyddzpkindyyg', '130', '802')]


testStatic1MB = [('leerqakjvtjuzayvzf', '1', '305'), ('leerqakjvtjuzayvzf', '1', '64'), ('leerqakjvtjuzayvzf', '1', '104'), ('leerqakjvtjuzayvzf', '1', '107'), ('leerqakjvtjuzayvzf', '1', '98'), ('leerqakjvtjuzayvzf', '1', '78'), ('leerqakjvtjuzayvzf', '1', '54'), ('leerqakjvtjuzayvzf', '1', '55'), ('leerqakjvtjuzayvzf', '1', '90'), ('leerqakjvtjuzayvzf', '1', '105'), ('leerqakjvtjuzayvzf', '1', '199'), ('leerqakjvtjuzayvzf', '1', '122'), ('leerqakjvtjuzayvzf', '1', '122'), ('leerqakjvtjuzayvzf', '1', '126'), ('leerqakjvtjuzayvzf', '1', '141'), ('leerqakjvtjuzayvzf', '1', '133'), ('leerqakjvtjuzayvzf', '1', '106'), ('leerqakjvtjuzayvzf', '1', '100'), ('leerqakjvtjuzayvzf', '1', '163'), ('leerqakjvtjuzayvzf', '1', '125'), ('leerqakjvtjuzayvzf', '1', '140'), ('leerqakjvtjuzayvzf', '1', '123'), ('leerqakjvtjuzayvzf', '1', '89'), ('leerqakjvtjuzayvzf', '1', '117'), ('leerqakjvtjuzayvzf', '1', '115'), ('leerqakjvtjuzayvzf', '1', '131'), ('leerqakjvtjuzayvzf', '1', '113'), ('leerqakjvtjuzayvzf', '1', '129'), ('leerqakjvtjuzayvzf', '1', '119'), ('leerqakjvtjuzayvzf', '1', '125'), ('leerqakjvtjuzayvzf', '1', '125'), ('leerqakjvtjuzayvzf', '1', '121'), ('leerqakjvtjuzayvzf', '1', '116'), ('leerqakjvtjuzayvzf', '1', '125'), ('leerqakjvtjuzayvzf', '1', '116'), ('leerqakjvtjuzayvzf', '1', '110'), ('leerqakjvtjuzayvzf', '1', '105'), ('leerqakjvtjuzayvzf', '1', '121'), ('leerqakjvtjuzayvzf', '1', '109'), ('leerqakjvtjuzayvzf', '1', '119'), ('leerqakjvtjuzayvzf', '1', '116'), ('leerqakjvtjuzayvzf', '1', '119'), ('leerqakjvtjuzayvzf', '1', '115'), ('leerqakjvtjuzayvzf', '1', '114'), ('leerqakjvtjuzayvzf', '1', '115'), ('leerqakjvtjuzayvzf', '1', '188'), ('leerqakjvtjuzayvzf', '1', '115'), ('leerqakjvtjuzayvzf', '1', '117'), ('leerqakjvtjuzayvzf', '1', '111'), ('leerqakjvtjuzayvzf', '1', '114')]
testStatic10MB = [('gavqvrefisiskqwbdi', '13', '624'), ('gavqvrefisiskqwbdi', '13', '121'), ('gavqvrefisiskqwbdi', '13', '126'), ('gavqvrefisiskqwbdi', '13', '134'), ('gavqvrefisiskqwbdi', '13', '142'), ('gavqvrefisiskqwbdi', '13', '125'), ('gavqvrefisiskqwbdi', '13', '123'), ('gavqvrefisiskqwbdi', '13', '139'), ('gavqvrefisiskqwbdi', '13', '122'), ('gavqvrefisiskqwbdi', '13', '127'), ('gavqvrefisiskqwbdi', '13', '126'), ('gavqvrefisiskqwbdi', '13', '124'), ('gavqvrefisiskqwbdi', '13', '112'), ('gavqvrefisiskqwbdi', '13', '125'), ('gavqvrefisiskqwbdi', '13', '117'), ('gavqvrefisiskqwbdi', '13', '123'), ('gavqvrefisiskqwbdi', '13', '103'), ('gavqvrefisiskqwbdi', '13', '194'), ('gavqvrefisiskqwbdi', '13', '169'), ('gavqvrefisiskqwbdi', '13', '186'), ('gavqvrefisiskqwbdi', '13', '146'), ('gavqvrefisiskqwbdi', '13', '137'), ('gavqvrefisiskqwbdi', '13', '161'), ('gavqvrefisiskqwbdi', '13', '161'), ('gavqvrefisiskqwbdi', '13', '122'), ('gavqvrefisiskqwbdi', '13', '134'), ('gavqvrefisiskqwbdi', '13', '130'), ('gavqvrefisiskqwbdi', '13', '143'), ('gavqvrefisiskqwbdi', '13', '121'), ('gavqvrefisiskqwbdi', '13', '136'), ('gavqvrefisiskqwbdi', '13', '124'), ('gavqvrefisiskqwbdi', '13', '125'), ('gavqvrefisiskqwbdi', '13', '118'), ('gavqvrefisiskqwbdi', '13', '146'), ('gavqvrefisiskqwbdi', '13', '132'), ('gavqvrefisiskqwbdi', '13', '127'), ('gavqvrefisiskqwbdi', '13', '127'), ('gavqvrefisiskqwbdi', '13', '114'), ('gavqvrefisiskqwbdi', '13', '96'), ('gavqvrefisiskqwbdi', '13', '128'), ('gavqvrefisiskqwbdi', '13', '58'), ('gavqvrefisiskqwbdi', '13', '71'), ('gavqvrefisiskqwbdi', '13', '59'), ('gavqvrefisiskqwbdi', '13', '59'), ('gavqvrefisiskqwbdi', '13', '61'), ('gavqvrefisiskqwbdi', '13', '61'), ('gavqvrefisiskqwbdi', '13', '59'), ('gavqvrefisiskqwbdi', '13', '90'), ('gavqvrefisiskqwbdi', '13', '120'), ('gavqvrefisiskqwbdi', '13', '180')]
testStatic50MB = [('idvbxqtgwnebfwjqfa', '65', '338'), ('idvbxqtgwnebfwjqfa', '65', '64'), ('idvbxqtgwnebfwjqfa', '64', '116'), ('idvbxqtgwnebfwjqfa', '65', '91'), ('idvbxqtgwnebfwjqfa', '64', '79'), ('idvbxqtgwnebfwjqfa', '64', '56'), ('idvbxqtgwnebfwjqfa', '64', '113'), ('idvbxqtgwnebfwjqfa', '64', '54'), ('idvbxqtgwnebfwjqfa', '65', '88'), ('idvbxqtgwnebfwjqfa', '64', '58'), ('idvbxqtgwnebfwjqfa', '64', '69'), ('idvbxqtgwnebfwjqfa', '65', '90'), ('idvbxqtgwnebfwjqfa', '64', '166'), ('idvbxqtgwnebfwjqfa', '65', '114'), ('idvbxqtgwnebfwjqfa', '65', '95'), ('idvbxqtgwnebfwjqfa', '64', '118'), ('idvbxqtgwnebfwjqfa', '64', '192'), ('idvbxqtgwnebfwjqfa', '64', '136'), ('idvbxqtgwnebfwjqfa', '65', '52'), ('idvbxqtgwnebfwjqfa', '64', '55'), ('idvbxqtgwnebfwjqfa', '64', '66'), ('idvbxqtgwnebfwjqfa', '64', '57'), ('idvbxqtgwnebfwjqfa', '64', '114'), ('idvbxqtgwnebfwjqfa', '64', '52'), ('idvbxqtgwnebfwjqfa', '64', '52'), ('idvbxqtgwnebfwjqfa', '64', '52'), ('idvbxqtgwnebfwjqfa', '64', '96'), ('idvbxqtgwnebfwjqfa', '64', '59'), ('idvbxqtgwnebfwjqfa', '64', '59'), ('idvbxqtgwnebfwjqfa', '65', '59'), ('idvbxqtgwnebfwjqfa', '64', '59'), ('idvbxqtgwnebfwjqfa', '64', '59'), ('idvbxqtgwnebfwjqfa', '64', '56'), ('idvbxqtgwnebfwjqfa', '65', '57'), ('idvbxqtgwnebfwjqfa', '64', '57'), ('idvbxqtgwnebfwjqfa', '64', '51'), ('idvbxqtgwnebfwjqfa', '64', '52'), ('idvbxqtgwnebfwjqfa', '64', '58'), ('idvbxqtgwnebfwjqfa', '64', '57'), ('idvbxqtgwnebfwjqfa', '64', '81'), ('idvbxqtgwnebfwjqfa', '64', '52'), ('idvbxqtgwnebfwjqfa', '64', '70'), ('idvbxqtgwnebfwjqfa', '65', '53'), ('idvbxqtgwnebfwjqfa', '64', '125'), ('idvbxqtgwnebfwjqfa', '64', '102'), ('idvbxqtgwnebfwjqfa', '65', '56'), ('idvbxqtgwnebfwjqfa', '64', '68'), ('idvbxqtgwnebfwjqfa', '64', '58'), ('idvbxqtgwnebfwjqfa', '64', '50'), ('idvbxqtgwnebfwjqfa', '64', '51')]
testStatic100MB = [('lubualngqdpwveexvi', '129', '1275'), ('lubualngqdpwveexvi', '129', '59'), ('lubualngqdpwveexvi', '129', '107'), ('lubualngqdpwveexvi', '129', '56'), ('lubualngqdpwveexvi', '129', '58'), ('lubualngqdpwveexvi', '129', '60'), ('lubualngqdpwveexvi', '129', '61'), ('lubualngqdpwveexvi', '129', '60'), ('lubualngqdpwveexvi', '129', '67'), ('lubualngqdpwveexvi', '129', '63'), ('lubualngqdpwveexvi', '129', '59'), ('lubualngqdpwveexvi', '129', '57'), ('lubualngqdpwveexvi', '129', '58'), ('lubualngqdpwveexvi', '129', '52'), ('lubualngqdpwveexvi', '129', '73'), ('lubualngqdpwveexvi', '129', '62'), ('lubualngqdpwveexvi', '129', '60'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '51'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '51'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '51'), ('lubualngqdpwveexvi', '129', '51'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '51'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '52'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '130', '51'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '52'), ('lubualngqdpwveexvi', '129', '51'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '50'), ('lubualngqdpwveexvi', '129', '50')]

testStatic20MB = [('jhuafyyhxsrqytgpja', '32', '1261'), ('jhuafyyhxsrqytgpja', '32', '71'), ('jhuafyyhxsrqytgpja', '32', '59'), ('jhuafyyhxsrqytgpja', '32', '81'), ('jhuafyyhxsrqytgpja', '32', '68'), ('jhuafyyhxsrqytgpja', '32', '90'), ('jhuafyyhxsrqytgpja', '32', '74'), ('jhuafyyhxsrqytgpja', '32', '60'), ('jhuafyyhxsrqytgpja', '32', '86'), ('jhuafyyhxsrqytgpja', '32', '77'), ('jhuafyyhxsrqytgpja', '32', '81'), ('jhuafyyhxsrqytgpja', '32', '68'), ('jhuafyyhxsrqytgpja', '32', '81'), ('jhuafyyhxsrqytgpja', '32', '68'), ('jhuafyyhxsrqytgpja', '32', '62'), ('jhuafyyhxsrqytgpja', '32', '70'), ('jhuafyyhxsrqytgpja', '32', '60'), ('jhuafyyhxsrqytgpja', '32', '176'), ('jhuafyyhxsrqytgpja', '32', '95'), ('jhuafyyhxsrqytgpja', '32', '51'), ('jhuafyyhxsrqytgpja', '32', '75'), ('jhuafyyhxsrqytgpja', '32', '72'), ('jhuafyyhxsrqytgpja', '32', '68'), ('jhuafyyhxsrqytgpja', '32', '78'), ('jhuafyyhxsrqytgpja', '32', '69'), ('jhuafyyhxsrqytgpja', '32', '83'), ('jhuafyyhxsrqytgpja', '32', '74'), ('jhuafyyhxsrqytgpja', '32', '50'), ('jhuafyyhxsrqytgpja', '32', '72'), ('jhuafyyhxsrqytgpja', '32', '54'), ('jhuafyyhxsrqytgpja', '32', '76'), ('jhuafyyhxsrqytgpja', '32', '70'), ('jhuafyyhxsrqytgpja', '32', '81'), ('jhuafyyhxsrqytgpja', '32', '76'), ('jhuafyyhxsrqytgpja', '32', '73'), ('jhuafyyhxsrqytgpja', '32', '77'), ('jhuafyyhxsrqytgpja', '32', '68'), ('jhuafyyhxsrqytgpja', '32', '69'), ('jhuafyyhxsrqytgpja', '32', '68'), ('jhuafyyhxsrqytgpja', '32', '51'), ('jhuafyyhxsrqytgpja', '32', '76'), ('jhuafyyhxsrqytgpja', '32', '69'), ('jhuafyyhxsrqytgpja', '32', '71'), ('jhuafyyhxsrqytgpja', '32', '72'), ('jhuafyyhxsrqytgpja', '32', '120'), ('jhuafyyhxsrqytgpja', '32', '71'), ('jhuafyyhxsrqytgpja', '32', '74'), ('jhuafyyhxsrqytgpja', '32', '72'), ('jhuafyyhxsrqytgpja', '32', '92'), ('jhuafyyhxsrqytgpja', '32', '74')]



testStatic1MB = [('eabeccbaeceddbacae', ['329', '265', '275', '3971', '250', '186', '37273', '154', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['54', '47', '67', '917', '45', '40', '5134', '33', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['48', '42', '51', '793', '38', '39', '6135', '50', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['57', '47', '56', '781', '39', '55', '4912', '33', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['58', '44', '62', '856', '40', '37', '6388', '36', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['64', '53', '56', '885', '40', '37', '5475', '30', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['53', '44', '56', '753', '36', '36', '5652', '48', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['98', '65', '71', '825', '37', '39', '5039', '36', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['64', '48', '56', '846', '41', '40', '4819', '41', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['57', '50', '55', '755', '39', '38', '4948', '56', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['65', '46', '64', '817', '37', '38', '5385', '29', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['68', '87', '63', '834', '38', '42', '4494', '34', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['56', '47', '60', '1247', '59', '43', '5436', '40', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['58', '47', '75', '919', '60', '42', '4933', '33', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['55', '68', '67', '920', '55', '57', '5942', '35', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['67', '64', '62', '815', '39', '40', '5399', '36', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['60', '54', '62', '777', '36', '36', '4605', '29', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['53', '43', '53', '748', '36', '36', '5092', '29', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['51', '44', '53', '748', '36', '36', '5246', '33', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['59', '45', '58', '818', '42', '39', '5960', '51', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['56', '76', '67', '785', '40', '45', '4780', '32', '9', '1', '2', '5']), ('eabeccbaeceddbacae', ['55', '44', '53', '752', '38', '76', '4947', '30', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['52', '43', '54', '778', '37', '37', '4570', '29', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['49', '56', '60', '1105', '41', '41', '5358', '36', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['54', '44', '55', '770', '40', '49', '5480', '34', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['60', '49', '56', '804', '54', '54', '5887', '32', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['56', '44', '55', '774', '45', '43', '4732', '33', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['62', '47', '61', '841', '43', '39', '5018', '32', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['67', '76', '90', '823', '40', '41', '5648', '46', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['52', '45', '54', '943', '50', '46', '5645', '35', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['56', '48', '55', '865', '37', '38', '5844', '33', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['65', '54', '80', '984', '86', '49', '6741', '37', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['60', '49', '57', '943', '45', '45', '5117', '32', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['57', '46', '59', '775', '41', '40', '6181', '34', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['59', '55', '72', '840', '43', '40', '4960', '32', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['58', '46', '65', '777', '37', '73', '5494', '33', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['67', '55', '64', '994', '43', '45', '4868', '36', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['60', '47', '69', '794', '40', '38', '4871', '32', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['52', '44', '54', '755', '36', '38', '5809', '38', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['57', '46', '62', '803', '37', '44', '5301', '61', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['54', '67', '64', '786', '42', '41', '5453', '37', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['74', '69', '94', '1063', '87', '97', '12878', '46', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['62', '64', '74', '821', '38', '37', '7104', '43', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['68', '63', '70', '823', '37', '44', '5773', '42', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['56', '81', '71', '837', '37', '68', '4967', '39', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['60', '44', '57', '978', '42', '51', '4867', '29', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['51', '44', '51', '732', '56', '41', '5811', '41', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['96', '58', '68', '960', '41', '45', '7942', '34', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['58', '56', '66', '944', '41', '45', '9606', '36', '9', '1', '2', '4']), ('eabeccbaeceddbacae', ['181', '56', '85', '776', '44', '45', '4868', '87', '9', '1', '2', '4'])]
testStatic10MB = [('cdccbaaebecddcaecd', ['1485', '267', '262', '3295', '329', '252', '40262', '176', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['83', '58', '82', '1360', '63', '71', '7442', '118', '85', '12', '44', '51']), ('cdccbaaebecddcaecd', ['65', '45', '51', '829', '39', '38', '4474', '61', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['120', '59', '78', '914', '59', '42', '5899', '33', '85', '13', '44', '51']), ('cdccbaaebecddcaecd', ['67', '46', '54', '793', '37', '37', '8856', '37', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['65', '51', '56', '831', '45', '38', '4930', '32', '85', '12', '44', '51']), ('cdccbaaebecddcaecd', ['68', '49', '60', '843', '38', '38', '5007', '38', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['139', '50', '65', '1064', '48', '52', '6444', '39', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['97', '80', '99', '1210', '59', '53', '7577', '35', '85', '13', '44', '51']), ('cdccbaaebecddcaecd', ['86', '61', '74', '1075', '42', '49', '5328', '34', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['70', '45', '58', '1517', '60', '54', '10373', '37', '85', '13', '43', '51']), ('cdccbaaebecddcaecd', ['57', '43', '55', '850', '47', '46', '8383', '34', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['70', '55', '53', '754', '36', '38', '7821', '33', '85', '12', '44', '51']), ('cdccbaaebecddcaecd', ['64', '46', '51', '728', '35', '35', '5347', '31', '87', '12', '44', '51']), ('cdccbaaebecddcaecd', ['90', '44', '58', '1042', '49', '48', '5610', '30', '85', '12', '43', '51']), ('cdccbaaebecddcaecd', ['73', '50', '57', '1300', '62', '52', '6470', '41', '86', '13', '44', '51']), ('cdccbaaebecddcaecd', ['71', '46', '75', '849', '42', '40', '4997', '39', '85', '12', '44', '51']), ('cdccbaaebecddcaecd', ['72', '43', '52', '744', '35', '37', '4459', '36', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['73', '54', '98', '861', '46', '45', '7749', '74', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['100', '56', '71', '1037', '48', '50', '6627', '42', '87', '12', '44', '51']), ('cdccbaaebecddcaecd', ['110', '67', '68', '1010', '51', '48', '7037', '41', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['88', '78', '72', '1080', '48', '47', '9127', '63', '85', '12', '44', '51']), ('cdccbaaebecddcaecd', ['121', '83', '151', '1279', '41', '39', '8620', '54', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['62', '43', '75', '764', '36', '43', '4653', '54', '85', '13', '44', '51']), ('cdccbaaebecddcaecd', ['88', '63', '102', '1102', '47', '37', '5220', '31', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['62', '47', '62', '1003', '53', '91', '9318', '49', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['76', '51', '91', '1594', '39', '58', '7829', '42', '85', '13', '44', '51']), ('cdccbaaebecddcaecd', ['97', '70', '85', '1266', '62', '61', '7976', '48', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['62', '49', '54', '803', '38', '36', '4549', '35', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['66', '47', '75', '1184', '49', '40', '4840', '30', '86', '13', '44', '51']), ('cdccbaaebecddcaecd', ['61', '43', '58', '768', '36', '40', '5535', '30', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['96', '66', '120', '876', '49', '67', '6487', '52', '85', '12', '44', '51']), ('cdccbaaebecddcaecd', ['134', '106', '61', '770', '67', '45', '9224', '39', '85', '12', '44', '51']), ('cdccbaaebecddcaecd', ['89', '59', '69', '1018', '45', '49', '6428', '39', '85', '13', '44', '51']), ('cdccbaaebecddcaecd', ['100', '54', '67', '991', '44', '44', '6258', '40', '87', '12', '44', '51']), ('cdccbaaebecddcaecd', ['79', '59', '99', '1007', '46', '57', '6672', '36', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['77', '61', '69', '1005', '50', '47', '6283', '42', '85', '12', '44', '51']), ('cdccbaaebecddcaecd', ['84', '52', '71', '989', '49', '48', '6707', '42', '86', '13', '44', '51']), ('cdccbaaebecddcaecd', ['82', '56', '68', '1033', '48', '45', '6245', '39', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['93', '75', '85', '1083', '49', '51', '6707', '43', '85', '12', '44', '51']), ('cdccbaaebecddcaecd', ['79', '62', '67', '994', '49', '49', '6512', '39', '85', '13', '44', '51']), ('cdccbaaebecddcaecd', ['87', '56', '66', '1010', '47', '52', '6290', '37', '86', '13', '44', '51']), ('cdccbaaebecddcaecd', ['99', '62', '146', '1547', '124', '93', '8626', '56', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['86', '60', '69', '1072', '51', '58', '8146', '60', '87', '12', '44', '51']), ('cdccbaaebecddcaecd', ['84', '65', '72', '1034', '46', '47', '6775', '43', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['70', '53', '104', '1040', '39', '45', '12445', '104', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['111', '63', '106', '1569', '55', '53', '15636', '60', '87', '12', '44', '51']), ('cdccbaaebecddcaecd', ['64', '43', '63', '760', '37', '44', '5457', '33', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['74', '48', '64', '756', '72', '99', '9042', '29', '86', '12', '44', '51']), ('cdccbaaebecddcaecd', ['68', '66', '55', '1028', '38', '35', '5183', '30', '86', '13', '44', '51'])]
testStatic20MB = [('aebcbebaedabdeadac', ['499', '238', '234', '3461', '183', '173', '39371', '175', '169', '25', '99', '89']), ('aebcbebaedabdeadac', ['84', '47', '63', '1049', '36', '39', '6733', '36', '170', '25', '100', '89']), ('aebcbebaedabdeadac', ['90', '47', '62', '1012', '46', '45', '6795', '32', '170', '25', '99', '89']), ('aebcbebaedabdeadac', ['115', '65', '51', '1107', '47', '39', '4245', '29', '171', '25', '99', '89']), ('aebcbebaedabdeadac', ['63', '41', '83', '1755', '36', '48', '5800', '29', '171', '25', '99', '89']), ('aebcbebaedabdeadac', ['63', '49', '50', '727', '41', '35', '4305', '33', '170', '25', '99', '89']), ('aebcbebaedabdeadac', ['71', '41', '50', '738', '35', '40', '4275', '29', '169', '25', '100', '89']), ('aebcbebaedabdeadac', ['64', '41', '51', '739', '35', '35', '7568', '40', '170', '25', '99', '89']), ('aebcbebaedabdeadac', ['85', '47', '61', '950', '44', '41', '6117', '33', '169', '25', '99', '89']), ('aebcbebaedabdeadac', ['86', '52', '69', '1000', '42', '39', '6296', '34', '170', '25', '100', '89']), ('aebcbebaedabdeadac', ['89', '54', '74', '997', '47', '44', '6628', '34', '170', '25', '98', '89']), ('aebcbebaedabdeadac', ['126', '56', '114', '1144', '44', '41', '6113', '33', '174', '25', '99', '89']), ('aebcbebaedabdeadac', ['88', '51', '61', '1000', '45', '40', '6400', '35', '169', '25', '99', '89']), ('aebcbebaedabdeadac', ['99', '53', '59', '970', '41', '38', '6302', '34', '170', '25', '99', '89']), ('aebcbebaedabdeadac', ['84', '47', '62', '969', '42', '41', '8069', '38', '171', '25', '99', '89']), ('aebcbebaedabdeadac', ['69', '41', '59', '748', '35', '35', '4498', '31', '170', '25', '99', '89']), ('aebcbebaedabdeadac', ['66', '42', '52', '766', '36', '36', '4595', '29', '170', '25', '99', '89']), ('aebcbebaedabdeadac', ['65', '42', '52', '765', '36', '36', '4589', '29', '169', '25', '98', '89']), ('aebcbebaedabdeadac', ['66', '42', '52', '759', '36', '35', '4598', '29', '169', '25', '100', '89']), ('aebcbebaedabdeadac', ['91', '53', '79', '1293', '48', '55', '7043', '40', '170', '25', '100', '89']), ('aebcbebaedabdeadac', ['85', '51', '59', '999', '41', '41', '6385', '33', '170', '25', '100', '89']), ('aebcbebaedabdeadac', ['91', '56', '58', '1019', '45', '46', '7356', '35', '170', '25', '99', '89']), ('aebcbebaedabdeadac', ['87', '48', '60', '956', '42', '42', '6141', '34', '169', '25', '100', '89']), ('aebcbebaedabdeadac', ['86', '47', '67', '999', '45', '43', '6348', '33', '169', '25', '99', '89']), ('aebcbebaedabdeadac', ['87', '48', '61', '942', '44', '42', '6191', '37', '169', '25', '99', '89']), ('aebcbebaedabdeadac', ['87', '50', '65', '991', '42', '43', '6771', '38', '170', '25', '100', '89']), ('aebcbebaedabdeadac', ['84', '48', '61', '941', '40', '40', '6306', '34', '171', '25', '100', '89']), ('aebcbebaedabdeadac', ['100', '53', '60', '969', '43', '43', '6077', '34', '173', '25', '99', '89']), ('aebcbebaedabdeadac', ['83', '48', '59', '982', '46', '40', '4915', '30', '169', '25', '99', '89']), ('aebcbebaedabdeadac', ['67', '42', '86', '1017', '48', '44', '6704', '41', '171', '25', '99', '89']), ('aebcbebaedabdeadac', ['87', '48', '61', '978', '41', '42', '6130', '36', '170', '25', '100', '89']), ('aebcbebaedabdeadac', ['85', '52', '62', '959', '41', '42', '6303', '35', '169', '26', '100', '89']), ('aebcbebaedabdeadac', ['91', '54', '59', '922', '40', '43', '6271', '33', '171', '25', '98', '89']), ('aebcbebaedabdeadac', ['81', '47', '64', '958', '41', '40', '6298', '33', '171', '25', '98', '89']), ('aebcbebaedabdeadac', ['88', '55', '78', '1010', '46', '43', '6355', '35', '170', '25', '100', '89']), ('aebcbebaedabdeadac', ['88', '51', '64', '948', '45', '42', '6680', '39', '169', '25', '99', '89']), ('aebcbebaedabdeadac', ['86', '52', '67', '993', '45', '40', '6136', '36', '169', '25', '99', '89']), ('aebcbebaedabdeadac', ['89', '50', '65', '936', '45', '40', '6232', '34', '171', '25', '100', '89']), ('aebcbebaedabdeadac', ['81', '53', '63', '977', '42', '42', '6056', '33', '170', '25', '100', '89']), ('aebcbebaedabdeadac', ['83', '53', '67', '967', '51', '50', '6847', '32', '167', '25', '99', '89']), ('aebcbebaedabdeadac', ['99', '45', '52', '748', '36', '35', '6929', '40', '172', '25', '100', '89']), ('aebcbebaedabdeadac', ['66', '42', '51', '753', '35', '39', '4367', '29', '169', '25', '99', '89']), ('aebcbebaedabdeadac', ['64', '41', '51', '749', '35', '35', '4427', '29', '169', '25', '99', '89']), ('aebcbebaedabdeadac', ['65', '44', '52', '743', '35', '35', '4453', '29', '171', '25', '98', '89']), ('aebcbebaedabdeadac', ['99', '41', '51', '744', '35', '36', '4398', '29', '170', '25', '99', '89']), ('aebcbebaedabdeadac', ['67', '42', '50', '741', '36', '35', '4499', '29', '172', '25', '98', '89']), ('aebcbebaedabdeadac', ['65', '42', '52', '959', '62', '58', '7659', '28', '168', '25', '99', '89']), ('aebcbebaedabdeadac', ['81', '46', '51', '761', '36', '36', '4336', '29', '172', '25', '99', '89']), ('aebcbebaedabdeadac', ['68', '98', '66', '1381', '48', '60', '4752', '31', '171', '25', '99', '89']), ('aebcbebaedabdeadac', ['86', '53', '49', '1094', '38', '36', '4611', '32', '170', '25', '99', '89'])]
testStatic30MB = [('abecabbadceebcbbdc', ['1282', '248', '242', '3593', '188', '185', '39794', '203', '252', '38', '149', '137']), ('abecabbadceebcbbdc', ['60', '44', '51', '761', '38', '68', '8181', '32', '253', '38', '147', '137']), ('abecabbadceebcbbdc', ['66', '48', '58', '942', '41', '41', '6168', '33', '250', '38', '147', '137']), ('abecabbadceebcbbdc', ['75', '49', '57', '963', '40', '39', '6023', '34', '254', '38', '147', '137']), ('abecabbadceebcbbdc', ['75', '49', '57', '935', '40', '41', '6100', '32', '253', '39', '148', '137']), ('abecabbadceebcbbdc', ['60', '44', '63', '825', '36', '36', '4529', '30', '253', '38', '150', '137']), ('abecabbadceebcbbdc', ['66', '43', '54', '792', '36', '36', '4717', '29', '253', '38', '149', '137']), ('abecabbadceebcbbdc', ['60', '49', '53', '764', '49', '36', '5174', '61', '253', '38', '149', '137']), ('abecabbadceebcbbdc', ['71', '55', '57', '931', '41', '39', '6571', '34', '254', '39', '148', '137']), ('abecabbadceebcbbdc', ['60', '46', '52', '1005', '36', '42', '5119', '30', '253', '38', '147', '137']), ('abecabbadceebcbbdc', ['65', '43', '59', '781', '35', '39', '4426', '31', '253', '38', '151', '137']), ('abecabbadceebcbbdc', ['62', '46', '50', '999', '35', '35', '5339', '69', '256', '38', '150', '137']), ('abecabbadceebcbbdc', ['60', '44', '52', '755', '39', '53', '7494', '35', '252', '38', '148', '137']), ('abecabbadceebcbbdc', ['73', '50', '56', '919', '43', '40', '5983', '34', '253', '38', '148', '137']), ('abecabbadceebcbbdc', ['90', '56', '51', '1084', '36', '35', '10371', '48', '251', '38', '149', '137']), ('abecabbadceebcbbdc', ['60', '44', '51', '755', '36', '36', '4650', '30', '251', '38', '149', '137']), ('abecabbadceebcbbdc', ['60', '44', '57', '778', '36', '36', '4421', '29', '254', '38', '148', '137']), ('abecabbadceebcbbdc', ['66', '46', '64', '795', '36', '36', '4554', '30', '253', '38', '149', '137']), ('abecabbadceebcbbdc', ['60', '43', '62', '790', '36', '39', '5231', '30', '254', '38', '150', '137']), ('abecabbadceebcbbdc', ['156', '55', '60', '800', '41', '70', '4608', '30', '252', '38', '150', '137']), ('abecabbadceebcbbdc', ['64', '44', '53', '792', '40', '38', '4658', '34', '252', '38', '148', '137']), ('abecabbadceebcbbdc', ['60', '49', '51', '761', '39', '36', '5021', '30', '252', '38', '147', '137']), ('abecabbadceebcbbdc', ['62', '47', '51', '773', '36', '39', '5834', '30', '254', '38', '150', '137']), ('abecabbadceebcbbdc', ['60', '44', '52', '824', '62', '44', '5963', '36', '252', '38', '149', '137']), ('abecabbadceebcbbdc', ['72', '44', '55', '781', '36', '36', '4723', '56', '254', '39', '149', '137']), ('abecabbadceebcbbdc', ['77', '48', '57', '963', '41', '39', '5931', '32', '252', '38', '148', '137']), ('abecabbadceebcbbdc', ['76', '48', '57', '928', '41', '43', '5953', '32', '252', '39', '149', '137']), ('abecabbadceebcbbdc', ['74', '49', '61', '923', '39', '59', '5849', '29', '252', '38', '149', '137']), ('abecabbadceebcbbdc', ['61', '44', '52', '772', '39', '36', '7391', '36', '252', '38', '149', '137']), ('abecabbadceebcbbdc', ['79', '52', '62', '949', '41', '101', '6484', '53', '254', '38', '150', '137']), ('abecabbadceebcbbdc', ['60', '43', '52', '754', '45', '47', '4422', '29', '252', '38', '149', '137']), ('abecabbadceebcbbdc', ['60', '44', '51', '858', '54', '36', '5232', '35', '255', '38', '150', '137']), ('abecabbadceebcbbdc', ['76', '51', '61', '962', '41', '42', '6401', '33', '254', '38', '148', '137']), ('abecabbadceebcbbdc', ['67', '102', '96', '1369', '68', '67', '6397', '33', '253', '38', '149', '137']), ('abecabbadceebcbbdc', ['74', '52', '60', '964', '37', '39', '6534', '36', '253', '38', '148', '137']), ('abecabbadceebcbbdc', ['71', '52', '60', '945', '43', '39', '6203', '32', '255', '38', '148', '137']), ('abecabbadceebcbbdc', ['74', '49', '58', '948', '41', '42', '6190', '34', '255', '38', '148', '137']), ('abecabbadceebcbbdc', ['85', '51', '60', '957', '43', '45', '6360', '33', '254', '38', '150', '137']), ('abecabbadceebcbbdc', ['121', '44', '51', '894', '54', '45', '4753', '30', '253', '38', '149', '137']), ('abecabbadceebcbbdc', ['63', '44', '51', '749', '36', '39', '8043', '36', '252', '38', '149', '137']), ('abecabbadceebcbbdc', ['75', '48', '59', '959', '41', '41', '6134', '33', '253', '38', '148', '137']), ('abecabbadceebcbbdc', ['60', '43', '65', '1242', '44', '50', '7550', '37', '253', '38', '149', '137']), ('abecabbadceebcbbdc', ['148', '86', '103', '946', '48', '41', '6508', '33', '254', '38', '148', '137']), ('abecabbadceebcbbdc', ['58', '43', '50', '1050', '78', '37', '7974', '32', '253', '38', '147', '137']), ('abecabbadceebcbbdc', ['69', '47', '56', '887', '39', '45', '6199', '33', '255', '38', '147', '137']), ('abecabbadceebcbbdc', ['70', '49', '58', '920', '41', '51', '6555', '33', '254', '38', '147', '137']), ('abecabbadceebcbbdc', ['60', '43', '54', '777', '37', '36', '4405', '30', '255', '38', '148', '137']), ('abecabbadceebcbbdc', ['59', '43', '51', '760', '36', '36', '4395', '38', '255', '38', '150', '137']), ('abecabbadceebcbbdc', ['67', '50', '51', '955', '38', '36', '5506', '30', '251', '38', '149', '137']), ('abecabbadceebcbbdc', ['61', '44', '57', '826', '66', '55', '5541', '32', '255', '38', '149', '137'])]
testStatic50MB = [('bebedeecdabeccbcee', ['441', '264', '248', '3357', '192', '276', '52234', '336', '421', '64', '227', '259']), ('bebedeecdabeccbcee', ['80', '47', '60', '947', '42', '40', '7202', '33', '420', '64', '230', '259']), ('bebedeecdabeccbcee', ['66', '70', '59', '933', '59', '38', '5906', '36', '424', '64', '227', '259']), ('bebedeecdabeccbcee', ['64', '49', '59', '1040', '40', '43', '6100', '33', '422', '64', '229', '259']), ('bebedeecdabeccbcee', ['115', '76', '117', '1761', '64', '60', '10639', '41', '421', '64', '230', '259']), ('bebedeecdabeccbcee', ['108', '68', '95', '1450', '61', '59', '10619', '41', '423', '64', '223', '259']), ('bebedeecdabeccbcee', ['58', '43', '51', '778', '39', '37', '5077', '31', '420', '64', '228', '259']), ('bebedeecdabeccbcee', ['67', '47', '57', '1641', '40', '61', '6627', '30', '427', '64', '228', '259']), ('bebedeecdabeccbcee', ['65', '54', '68', '879', '52', '56', '14005', '69', '420', '64', '231', '259']), ('bebedeecdabeccbcee', ['146', '71', '122', '1531', '81', '72', '12902', '54', '415', '64', '226', '259']), ('bebedeecdabeccbcee', ['66', '48', '66', '989', '42', '42', '6504', '37', '419', '64', '227', '259']), ('bebedeecdabeccbcee', ['68', '52', '63', '1109', '42', '43', '6552', '35', '419', '64', '228', '259']), ('bebedeecdabeccbcee', ['71', '47', '60', '935', '40', '43', '6309', '33', '424', '64', '228', '259']), ('bebedeecdabeccbcee', ['108', '141', '88', '1549', '40', '41', '5943', '34', '419', '64', '228', '259']), ('bebedeecdabeccbcee', ['67', '47', '62', '935', '42', '41', '6198', '33', '424', '64', '227', '259']), ('bebedeecdabeccbcee', ['141', '129', '124', '2524', '122', '78', '12047', '49', '422', '64', '228', '259']), ('bebedeecdabeccbcee', ['98', '64', '75', '1403', '62', '65', '12363', '51', '423', '65', '228', '259']), ('bebedeecdabeccbcee', ['223', '54', '69', '809', '38', '38', '4814', '31', '418', '64', '225', '259']), ('bebedeecdabeccbcee', ['58', '54', '179', '2328', '133', '88', '15918', '60', '420', '64', '230', '259']), ('bebedeecdabeccbcee', ['125', '44', '54', '1132', '36', '38', '5206', '37', '420', '64', '223', '259']), ('bebedeecdabeccbcee', ['114', '73', '132', '1756', '62', '65', '12188', '51', '423', '64', '228', '259']), ('bebedeecdabeccbcee', ['65', '49', '61', '1178', '44', '42', '6551', '34', '423', '64', '229', '259']), ('bebedeecdabeccbcee', ['57', '46', '52', '783', '37', '36', '10813', '30', '422', '64', '229', '259']), ('bebedeecdabeccbcee', ['71', '59', '56', '864', '78', '43', '4643', '30', '424', '64', '227', '259']), ('bebedeecdabeccbcee', ['58', '86', '191', '1335', '84', '110', '10258', '33', '419', '64', '228', '259']), ('bebedeecdabeccbcee', ['61', '45', '61', '1003', '41', '44', '7331', '52', '422', '64', '228', '259']), ('bebedeecdabeccbcee', ['113', '51', '61', '1019', '46', '84', '4719', '30', '419', '64', '229', '259']), ('bebedeecdabeccbcee', ['58', '43', '55', '981', '36', '36', '4417', '30', '422', '64', '227', '259']), ('bebedeecdabeccbcee', ['63', '44', '51', '853', '37', '40', '4493', '30', '422', '64', '228', '259']), ('bebedeecdabeccbcee', ['58', '43', '51', '1030', '36', '47', '6237', '33', '420', '64', '227', '259']), ('bebedeecdabeccbcee', ['65', '48', '59', '990', '73', '36', '4608', '30', '420', '64', '229', '259']), ('bebedeecdabeccbcee', ['76', '50', '61', '1158', '40', '39', '6210', '33', '425', '64', '229', '259']), ('bebedeecdabeccbcee', ['64', '46', '59', '985', '43', '37', '4882', '31', '422', '64', '227', '259']), ('bebedeecdabeccbcee', ['66', '67', '59', '959', '40', '40', '5961', '30', '422', '64', '228', '259']), ('bebedeecdabeccbcee', ['94', '43', '52', '803', '38', '36', '6240', '39', '421', '64', '231', '259']), ('bebedeecdabeccbcee', ['66', '48', '61', '1045', '41', '47', '6748', '34', '423', '64', '231', '259']), ('bebedeecdabeccbcee', ['66', '50', '57', '994', '40', '42', '6210', '34', '420', '64', '225', '259']), ('bebedeecdabeccbcee', ['88', '49', '103', '795', '37', '38', '4914', '31', '424', '64', '226', '259']), ('bebedeecdabeccbcee', ['79', '51', '62', '1019', '43', '42', '6570', '34', '423', '64', '231', '259']), ('bebedeecdabeccbcee', ['71', '49', '186', '1085', '42', '43', '11805', '87', '421', '64', '226', '259']), ('bebedeecdabeccbcee', ['81', '53', '108', '1048', '37', '37', '5140', '30', '423', '64', '228', '259']), ('bebedeecdabeccbcee', ['68', '98', '90', '820', '37', '36', '8085', '77', '424', '64', '231', '259']), ('bebedeecdabeccbcee', ['83', '45', '53', '927', '51', '65', '10561', '37', '419', '64', '224', '259']), ('bebedeecdabeccbcee', ['61', '43', '54', '800', '38', '39', '9295', '30', '418', '64', '231', '259']), ('bebedeecdabeccbcee', ['69', '48', '60', '1121', '40', '87', '12458', '33', '420', '64', '229', '260']), ('bebedeecdabeccbcee', ['58', '44', '52', '809', '41', '38', '7008', '53', '419', '64', '229', '259']), ('bebedeecdabeccbcee', ['72', '45', '53', '952', '40', '36', '5307', '31', '423', '64', '227', '259']), ('bebedeecdabeccbcee', ['63', '43', '54', '911', '37', '37', '6343', '31', '417', '64', '224', '259']), ('bebedeecdabeccbcee', ['204', '146', '51', '762', '36', '36', '4846', '32', '419', '64', '227', '259']), ('bebedeecdabeccbcee', ['77', '115', '65', '2521', '105', '37', '5682', '30', '422', '64', '221', '259'])]
testStatic75MB = [('cbcabacdeccaeeeebb', ['2234', '223', '228', '2618', '169', '170', '32534', '159', '619', '96', '352', '385']), ('cbcabacdeccaeeeebb', ['59', '59', '53', '756', '37', '36', '4430', '33', '611', '96', '349', '385']), ('cbcabacdeccaeeeebb', ['80', '42', '53', '762', '36', '36', '4518', '30', '618', '96', '347', '385']), ('cbcabacdeccaeeeebb', ['61', '70', '52', '766', '39', '36', '4455', '30', '618', '96', '348', '385']), ('cbcabacdeccaeeeebb', ['58', '43', '53', '760', '37', '37', '4489', '30', '614', '96', '348', '385']), ('cbcabacdeccaeeeebb', ['56', '58', '52', '760', '37', '37', '4413', '30', '617', '96', '345', '385']), ('cbcabacdeccaeeeebb', ['65', '42', '52', '954', '43', '43', '6320', '33', '618', '96', '347', '385']), ('cbcabacdeccaeeeebb', ['173', '156', '63', '1008', '44', '43', '6501', '30', '628', '97', '342', '385']), ('cbcabacdeccaeeeebb', ['79', '66', '87', '903', '45', '45', '5684', '47', '613', '96', '345', '385']), ('cbcabacdeccaeeeebb', ['73', '70', '53', '976', '38', '58', '10040', '30', '626', '96', '347', '385']), ('cbcabacdeccaeeeebb', ['69', '41', '51', '833', '37', '121', '6266', '59', '619', '96', '345', '385']), ('cbcabacdeccaeeeebb', ['70', '49', '58', '988', '43', '44', '6634', '36', '618', '97', '346', '385']), ('cbcabacdeccaeeeebb', ['65', '49', '61', '1019', '46', '43', '6936', '55', '614', '96', '347', '385']), ('cbcabacdeccaeeeebb', ['98', '88', '61', '954', '55', '55', '5245', '59', '623', '96', '349', '385']), ('cbcabacdeccaeeeebb', ['73', '42', '55', '1078', '45', '37', '6367', '30', '616', '97', '343', '385']), ('cbcabacdeccaeeeebb', ['74', '45', '53', '793', '38', '37', '5039', '34', '616', '96', '348', '385']), ('cbcabacdeccaeeeebb', ['118', '50', '67', '792', '38', '38', '5906', '49', '621', '97', '347', '385']), ('cbcabacdeccaeeeebb', ['131', '47', '72', '910', '43', '104', '5149', '32', '621', '97', '343', '385']), ('cbcabacdeccaeeeebb', ['81', '49', '60', '829', '66', '37', '5484', '31', '625', '97', '345', '385']), ('cbcabacdeccaeeeebb', ['91', '60', '53', '1026', '63', '42', '6061', '34', '618', '97', '348', '385']), ('cbcabacdeccaeeeebb', ['79', '62', '59', '1014', '43', '41', '6033', '36', '618', '97', '349', '385']), ('cbcabacdeccaeeeebb', ['91', '49', '63', '999', '45', '110', '6176', '36', '620', '96', '345', '385']), ('cbcabacdeccaeeeebb', ['133', '47', '77', '935', '42', '41', '6519', '78', '616', '96', '345', '385']), ('cbcabacdeccaeeeebb', ['76', '93', '72', '1110', '37', '40', '4582', '31', '611', '96', '342', '385']), ('cbcabacdeccaeeeebb', ['78', '60', '76', '791', '44', '44', '6235', '31', '631', '97', '348', '385']), ('cbcabacdeccaeeeebb', ['87', '45', '54', '843', '38', '40', '5871', '31', '621', '97', '346', '385']), ('cbcabacdeccaeeeebb', ['71', '44', '53', '1057', '38', '37', '4560', '30', '614', '96', '347', '385']), ('cbcabacdeccaeeeebb', ['120', '43', '54', '787', '38', '39', '4818', '31', '622', '96', '350', '385']), ('cbcabacdeccaeeeebb', ['109', '42', '54', '817', '37', '37', '4737', '31', '612', '96', '347', '385']), ('cbcabacdeccaeeeebb', ['195', '42', '112', '993', '88', '38', '7864', '56', '615', '96', '348', '385']), ('cbcabacdeccaeeeebb', ['71', '43', '54', '1066', '37', '37', '6129', '30', '620', '97', '345', '385']), ('cbcabacdeccaeeeebb', ['161', '43', '53', '797', '38', '46', '5391', '43', '612', '97', '350', '385']), ('cbcabacdeccaeeeebb', ['72', '48', '63', '952', '38', '41', '5989', '34', '619', '97', '347', '385']), ('cbcabacdeccaeeeebb', ['70', '44', '51', '905', '37', '36', '4406', '33', '618', '96', '348', '385']), ('cbcabacdeccaeeeebb', ['59', '42', '53', '739', '37', '36', '6007', '31', '621', '96', '352', '385']), ('cbcabacdeccaeeeebb', ['67', '66', '63', '999', '44', '56', '8965', '34', '630', '97', '351', '385']), ('cbcabacdeccaeeeebb', ['93', '43', '52', '802', '38', '37', '4692', '30', '627', '96', '349', '385']), ('cbcabacdeccaeeeebb', ['77', '114', '74', '991', '43', '43', '6069', '33', '617', '97', '350', '385']), ('cbcabacdeccaeeeebb', ['74', '48', '59', '946', '40', '41', '7184', '30', '614', '96', '348', '385']), ('cbcabacdeccaeeeebb', ['136', '56', '50', '802', '38', '50', '5598', '40', '617', '96', '349', '385']), ('cbcabacdeccaeeeebb', ['155', '43', '55', '976', '46', '79', '5160', '31', '624', '97', '349', '385']), ('cbcabacdeccaeeeebb', ['72', '45', '55', '793', '36', '36', '4847', '30', '613', '96', '343', '385']), ('cbcabacdeccaeeeebb', ['82', '142', '59', '1300', '40', '39', '5352', '32', '623', '96', '352', '385']), ('cbcabacdeccaeeeebb', ['71', '60', '62', '757', '41', '42', '6085', '32', '611', '97', '348', '385']), ('cbcabacdeccaeeeebb', ['182', '44', '56', '783', '38', '37', '4612', '31', '640', '96', '349', '385']), ('cbcabacdeccaeeeebb', ['75', '74', '74', '1109', '46', '49', '6483', '29', '647', '97', '349', '385']), ('cbcabacdeccaeeeebb', ['100', '45', '55', '964', '48', '40', '6101', '34', '615', '97', '356', '385']), ('cbcabacdeccaeeeebb', ['90', '49', '57', '937', '39', '40', '5908', '34', '613', '97', '344', '385']), ('cbcabacdeccaeeeebb', ['71', '60', '59', '939', '43', '43', '5917', '32', '613', '97', '346', '385']), ('cbcabacdeccaeeeebb', ['127', '48', '57', '931', '41', '43', '5869', '32', '617', '97', '350', '385'])]
testStatic100MB = [('cdadccccacddeceacb', ['408', '258', '258', '3081', '162', '182', '30274', '151', '810', '129', '484', '455']), ('cdadccccacddeceacb', ['110', '42', '66', '747', '36', '37', '4231', '30', '814', '130', '486', '455']), ('cdadccccacddeceacb', ['54', '43', '51', '752', '37', '37', '4247', '30', '801', '129', '485', '455']), ('cdadccccacddeceacb', ['54', '42', '51', '746', '36', '46', '4191', '30', '812', '129', '484', '455']), ('cdadccccacddeceacb', ['111', '59', '51', '927', '37', '36', '4209', '30', '812', '129', '485', '455']), ('cdadccccacddeceacb', ['54', '42', '51', '743', '36', '36', '4283', '30', '807', '129', '484', '455']), ('cdadccccacddeceacb', ['56', '42', '51', '748', '36', '36', '4194', '39', '810', '129', '483', '455']), ('cdadccccacddeceacb', ['54', '42', '51', '765', '36', '36', '4234', '30', '798', '129', '485', '455']), ('cdadccccacddeceacb', ['54', '42', '52', '753', '36', '36', '4169', '33', '804', '130', '485', '455']), ('cdadccccacddeceacb', ['56', '42', '51', '746', '37', '36', '4210', '31', '804', '129', '483', '455']), ('cdadccccacddeceacb', ['56', '44', '51', '745', '42', '36', '4222', '30', '818', '129', '486', '455']), ('cdadccccacddeceacb', ['54', '44', '51', '747', '37', '36', '4195', '30', '807', '129', '479', '455']), ('cdadccccacddeceacb', ['55', '67', '51', '928', '37', '36', '4175', '30', '814', '129', '483', '455']), ('cdadccccacddeceacb', ['54', '42', '51', '748', '36', '36', '4175', '30', '800', '129', '485', '455']), ('cdadccccacddeceacb', ['55', '42', '51', '744', '37', '36', '4171', '30', '806', '129', '483', '455']), ('cdadccccacddeceacb', ['55', '42', '52', '931', '37', '36', '4230', '30', '813', '129', '484', '455']), ('cdadccccacddeceacb', ['55', '42', '51', '767', '36', '36', '4163', '30', '806', '129', '484', '455']), ('cdadccccacddeceacb', ['54', '43', '52', '741', '36', '36', '4186', '30', '816', '130', '482', '455']), ('cdadccccacddeceacb', ['55', '42', '52', '745', '37', '36', '4146', '30', '799', '129', '482', '455']), ('cdadccccacddeceacb', ['54', '52', '51', '750', '72', '37', '5051', '39', '808', '129', '482', '455']), ('cdadccccacddeceacb', ['102', '42', '51', '761', '36', '38', '4557', '31', '803', '129', '482', '455']), ('cdadccccacddeceacb', ['102', '42', '53', '742', '37', '36', '4401', '30', '812', '129', '486', '455']), ('cdadccccacddeceacb', ['54', '140', '52', '756', '37', '36', '4207', '30', '812', '129', '485', '455']), ('cdadccccacddeceacb', ['55', '43', '51', '745', '37', '36', '4179', '30', '812', '129', '486', '455']), ('cdadccccacddeceacb', ['55', '50', '52', '750', '36', '36', '4195', '30', '803', '129', '481', '455']), ('cdadccccacddeceacb', ['55', '43', '52', '741', '36', '36', '4186', '30', '800', '129', '484', '455']), ('cdadccccacddeceacb', ['56', '50', '58', '746', '36', '36', '4241', '30', '809', '129', '484', '455']), ('cdadccccacddeceacb', ['55', '60', '51', '770', '36', '39', '4184', '30', '804', '129', '483', '455']), ('cdadccccacddeceacb', ['55', '58', '51', '744', '37', '36', '4283', '31', '815', '129', '483', '455']), ('cdadccccacddeceacb', ['54', '136', '78', '748', '36', '37', '4172', '30', '810', '129', '481', '455']), ('cdadccccacddeceacb', ['54', '59', '51', '748', '36', '36', '4160', '30', '808', '129', '487', '455']), ('cdadccccacddeceacb', ['55', '53', '51', '746', '37', '36', '4190', '40', '817', '129', '483', '455']), ('cdadccccacddeceacb', ['54', '53', '51', '829', '36', '36', '4194', '30', '811', '129', '483', '455']), ('cdadccccacddeceacb', ['54', '62', '51', '747', '36', '37', '4181', '32', '807', '129', '489', '455']), ('cdadccccacddeceacb', ['54', '61', '51', '746', '36', '39', '4242', '30', '813', '129', '483', '456']), ('cdadccccacddeceacb', ['55', '43', '51', '746', '37', '65', '4173', '30', '810', '129', '487', '455']), ('cdadccccacddeceacb', ['54', '43', '51', '749', '36', '46', '4175', '30', '806', '129', '481', '455']), ('cdadccccacddeceacb', ['54', '42', '51', '744', '36', '36', '4200', '31', '809', '129', '485', '455']), ('cdadccccacddeceacb', ['54', '53', '51', '744', '36', '36', '4291', '30', '814', '129', '480', '455']), ('cdadccccacddeceacb', ['54', '77', '52', '744', '37', '36', '4245', '30', '802', '129', '484', '455']), ('cdadccccacddeceacb', ['55', '55', '55', '744', '37', '36', '4206', '30', '811', '129', '487', '455']), ('cdadccccacddeceacb', ['54', '42', '51', '754', '132', '36', '4187', '30', '808', '129', '483', '455']), ('cdadccccacddeceacb', ['55', '43', '51', '749', '36', '36', '4217', '30', '801', '129', '483', '455']), ('cdadccccacddeceacb', ['54', '64', '52', '741', '36', '36', '4210', '31', '804', '129', '483', '455']), ('cdadccccacddeceacb', ['55', '128', '52', '750', '37', '134', '4160', '30', '825', '129', '486', '455']), ('cdadccccacddeceacb', ['55', '43', '51', '750', '37', '37', '4231', '30', '808', '129', '484', '455']), ('cdadccccacddeceacb', ['55', '45', '51', '742', '140', '37', '4171', '30', '813', '129', '482', '455']), ('cdadccccacddeceacb', ['55', '42', '51', '926', '36', '36', '4222', '30', '801', '129', '484', '455']), ('cdadccccacddeceacb', ['54', '42', '51', '752', '36', '36', '4206', '136', '810', '129', '491', '455']), ('cdadccccacddeceacb', ['89', '54', '52', '828', '96', '38', '4233', '31', '812', '129', '483', '455'])]

test100MB = [('bccbbddddbdbeceada', ['1234', '197', '208', '152', '814', '129', '737', '97']), ('bccbbddddbdbeceada', ['52', '42', '30', '32', '816', '129', '728', '97']), ('bccbbddddbdbeceada', ['54', '42', '32', '31', '796', '129', '725', '97']), ('bccbbddddbdbeceada', ['50', '45', '33', '30', '824', '129', '756', '97']), ('bccbbddddbdbeceada', ['51', '41', '30', '30', '801', '129', '738', '97']), ('bccbbddddbdbeceada', ['50', '43', '34', '33', '809', '129', '739', '97']), ('bccbbddddbdbeceada', ['51', '48', '33', '31', '803', '129', '755', '97']), ('bccbbddddbdbeceada', ['50', '46', '30', '29', '805', '129', '735', '97']), ('bccbbddddbdbeceada', ['53', '67', '32', '30', '823', '129', '733', '97']), ('bccbbddddbdbeceada', ['81', '71', '38', '30', '833', '129', '739', '97']), ('bccbbddddbdbeceada', ['49', '47', '30', '30', '812', '129', '735', '97']), ('bccbbddddbdbeceada', ['58', '41', '31', '34', '812', '129', '738', '97']), ('bccbbddddbdbeceada', ['49', '41', '30', '30', '815', '129', '738', '97']), ('bccbbddddbdbeceada', ['48', '41', '30', '30', '810', '129', '735', '97']), ('bccbbddddbdbeceada', ['61', '42', '30', '30', '800', '129', '728', '97']), ('bccbbddddbdbeceada', ['49', '41', '66', '32', '812', '129', '735', '97']), ('bccbbddddbdbeceada', ['48', '41', '31', '30', '803', '129', '730', '97']), ('bccbbddddbdbeceada', ['50', '42', '30', '29', '806', '129', '731', '97']), ('bccbbddddbdbeceada', ['48', '42', '30', '29', '811', '129', '734', '97']), ('bccbbddddbdbeceada', ['53', '60', '32', '29', '824', '129', '731', '97']), ('bccbbddddbdbeceada', ['49', '41', '31', '29', '811', '129', '731', '97']), ('bccbbddddbdbeceada', ['48', '41', '30', '29', '828', '129', '738', '97']), ('bccbbddddbdbeceada', ['76', '41', '30', '38', '799', '129', '735', '97']), ('bccbbddddbdbeceada', ['53', '44', '30', '29', '802', '129', '756', '97']), ('bccbbddddbdbeceada', ['49', '41', '33', '32', '827', '129', '731', '97']), ('bccbbddddbdbeceada', ['56', '45', '33', '30', '800', '129', '762', '97']), ('bccbbddddbdbeceada', ['51', '44', '36', '32', '813', '129', '734', '97']), ('bccbbddddbdbeceada', ['53', '44', '32', '31', '824', '129', '738', '97']), ('bccbbddddbdbeceada', ['54', '44', '33', '31', '816', '129', '735', '97']), ('bccbbddddbdbeceada', ['101', '62', '42', '30', '810', '129', '738', '97']), ('bccbbddddbdbeceada', ['63', '51', '52', '30', '801', '130', '734', '97']), ('bccbbddddbdbeceada', ['70', '42', '31', '32', '818', '129', '735', '97']), ('bccbbddddbdbeceada', ['51', '41', '30', '41', '830', '129', '738', '97']), ('bccbbddddbdbeceada', ['49', '42', '30', '30', '817', '129', '736', '97']), ('bccbbddddbdbeceada', ['51', '45', '33', '31', '813', '129', '735', '97']), ('bccbbddddbdbeceada', ['100', '69', '32', '36', '815', '129', '749', '97']), ('bccbbddddbdbeceada', ['206', '70', '154', '50', '817', '129', '728', '97']), ('bccbbddddbdbeceada', ['125', '102', '59', '40', '813', '129', '736', '97']), ('bccbbddddbdbeceada', ['89', '69', '39', '56', '804', '130', '731', '97']), ('bccbbddddbdbeceada', ['70', '69', '39', '50', '807', '130', '730', '97']), ('bccbbddddbdbeceada', ['82', '68', '39', '38', '805', '129', '737', '97']), ('bccbbddddbdbeceada', ['147', '60', '30', '29', '806', '129', '731', '97']), ('bccbbddddbdbeceada', ['47', '41', '31', '29', '814', '129', '736', '97']), ('bccbbddddbdbeceada', ['47', '40', '30', '29', '813', '129', '741', '97']), ('bccbbddddbdbeceada', ['48', '41', '30', '29', '806', '129', '736', '97']), ('bccbbddddbdbeceada', ['48', '40', '30', '29', '832', '129', '740', '97']), ('bccbbddddbdbeceada', ['48', '41', '30', '29', '803', '129', '735', '97']), ('bccbbddddbdbeceada', ['48', '41', '30', '29', '810', '129', '729', '97']), ('bccbbddddbdbeceada', ['48', '40', '31', '29', '810', '129', '729', '97']), ('bccbbddddbdbeceada', ['48', '40', '30', '29', '805', '129', '729', '97'])]

# randomString(256*1024*1024)
p = randomString(16,False)
match(p)
test()

True